In [1]:
import wfdb
from wfdb import processing
import pandas as pd
import numpy as np
import re
import os
from datetime import timedelta

import requests
from bs4 import BeautifulSoup

import plotly.graph_objs as go
import plotly.offline as pyo

# Use the signal for filters
from scipy.signal import butter, filtfilt, iirnotch


import logging
logging.basicConfig(level=logging.DEBUG)

In [2]:
# Base path to download the patient overview
base_url = "https://physionet.org/content/i-care/2.1/training/"
records_url = base_url + "RECORDS"

# Base directory path for the PhysioNet database
physionet_db_path = "i-care/2.1/training"

patient_numbers = []

In [3]:
destination_folder = "data"
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

In [4]:
# Download the RECORDS file to get the list of patient numbers
response = requests.get(records_url)
if response.status_code == 200:
    # The RECORDS file is a plain text file, so split it by lines to get the patient numbers
    patient_numbers_html = response.text
else:
    print("Failed to download the RECORDS file.")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): physionet.org:443
DEBUG:urllib3.connectionpool:https://physionet.org:443 "GET /content/i-care/2.1/training/RECORDS HTTP/11" 200 None


In [5]:
# Parse the HTML content
soup = BeautifulSoup(patient_numbers_html, 'html.parser')

# Find the <pre> tag with class "plain" and the <code> tag within it
code_tag = soup.find('pre', {'class': 'plain'}).find('code')

# Extract the content of the <code> tag
patient_records = code_tag.text if code_tag else ''

# Split the content into individual records (one per line)
patient_records_list = patient_records.splitlines()

# Clean the list by removing any empty strings
patient_records_list = [record.strip() for record in patient_records_list if record.strip()]

# Clean up the patient numbers by removing 'training/' and the trailing '/'
patient_numbers = [record.split('/')[1] for record in patient_records_list]

# Print the cleaned patient numbers to verify
print(patient_numbers)

['0284', '0286', '0296', '0299', '0303', '0306', '0311', '0312', '0313', '0316', '0319', '0320', '0326', '0328', '0332', '0334', '0335', '0337', '0340', '0341', '0342', '0344', '0346', '0347', '0348', '0349', '0350', '0351', '0352', '0353', '0354', '0355', '0356', '0357', '0358', '0359', '0360', '0361', '0362', '0363', '0364', '0365', '0366', '0367', '0368', '0369', '0370', '0371', '0372', '0373', '0375', '0376', '0377', '0378', '0379', '0380', '0382', '0383', '0384', '0385', '0387', '0389', '0390', '0391', '0392', '0394', '0395', '0396', '0397', '0398', '0399', '0400', '0402', '0403', '0404', '0405', '0406', '0407', '0409', '0410', '0411', '0412', '0413', '0414', '0415', '0416', '0417', '0418', '0419', '0420', '0421', '0422', '0423', '0424', '0426', '0427', '0428', '0429', '0430', '0431', '0432', '0433', '0434', '0435', '0436', '0437', '0438', '0439', '0440', '0441', '0442', '0443', '0444', '0445', '0446', '0447', '0448', '0450', '0451', '0452', '0453', '0455', '0456', '0457', '0458',

In [6]:
# Define the total number of segments (24 hours in 5-minute intervals)
total_duration_hours = 24
total_segments = (total_duration_hours * 60) // 5

# Define the columns by interleaving `Mean_HR` and `HRV_SDNN` columns for each segment
columns = ["Patient_ID"] + [f"Segment_{i+1}_Mean_HR" if j % 2 == 0 else f"Segment_{i+1}_HRV_SDNN" for i in range(total_segments) for j in range(2)]
final_df = pd.DataFrame(columns=columns)



In [7]:
def apply_high_low_filter(signal, sampling_rate, utility_freq):
    # High-pass filter to remove baseline drift (cutoff frequency = 0.5 Hz)
    b_high, a_high = butter(1, 0.5 / (sampling_rate / 2), btype='high')
    filtered_signal = filtfilt(b_high, a_high, signal)

    # Low-pass filter to remove high-frequency noise (cutoff frequency = 50 Hz)
    b_low, a_low = butter(1, 50 / (250 / 2), btype='low')
    filtered_signal = filtfilt(b_low, a_low, filtered_signal)

    # Notch filter to remove utility frequency noise (e.g., 50 Hz or 60 Hz)
    Q = 30.0
    b_notch, a_notch = iirnotch(utility_freq, Q, sampling_rate)
    filtered_signal = filtfilt(b_notch, a_notch, filtered_signal)

    return filtered_signal

In [8]:
# Define function to calculate HR and HRV for each 5-minute segment
def calculate_hr_hrv(signal, sampling_rate, local_start_time, total_samples):
    # Initialize the XQRS detector and detect QRS complexes
    xqrs = processing.XQRS(sig=signal, fs=sampling_rate)
    xqrs.detect(learn=True, verbose=False)
    qrs_indices = np.array(xqrs.qrs_inds)  # Indices of detected R-peaks in sample points

    # Convert QRS indices to local time in seconds (relative to recording start)
    r_wave_times = qrs_indices / sampling_rate

    # Calculate the duration to the next full 5-minute boundary from the local start time
    first_segment_duration = timedelta(minutes=5) - timedelta(seconds=local_start_time.total_seconds() % 300)
    first_segment_duration_seconds = first_segment_duration.total_seconds()
    print("First Segment Duration (s):", first_segment_duration_seconds)

    # Calculate the total duration in seconds
    total_duration = total_samples / sampling_rate  # in seconds

    # Calculate remaining time for full 5-minute segments after the first segment
    remaining_time = total_duration - first_segment_duration_seconds
    num_segments = int(remaining_time // 300)  # Full 5-minute segments
    print(f"Remaining Time (s): {remaining_time}, Full 5-Minute Segments: {num_segments}")

    # Initialize lists to store HR and HRV values, and the start points for each segment
    hr_values = []
    hrv_values = []
    segment_start_indices = []  # Store start points in sample indices

    # Process the shortened first segment using first_segment_duration directly
    first_segment_indices = r_wave_times < first_segment_duration_seconds
    first_segment_rr_intervals = np.diff(r_wave_times[first_segment_indices]) * 1000  # Convert to milliseconds

    # Calculate HR and HRV for the first segment
    if len(first_segment_rr_intervals) > 0:
        mean_hr = int(round(processing.calc_mean_hr(first_segment_rr_intervals / 1000, rr_units="seconds")))
        hrv = int(round(np.std(first_segment_rr_intervals)))
    else:
        mean_hr = np.nan
        hrv = np.nan

    # Append results for the first segment and its start index in sample points
    hr_values.append(mean_hr)
    hrv_values.append(hrv)
    # First segment boundary should be the beginning of the recording in samples
    first_segment_start_index = 0
    segment_start_indices.append(first_segment_start_index)

    # Update current_time to the end of the first segment for subsequent segments
    current_time = first_segment_duration_seconds

    # Process remaining full 5-minute segments based on the relative recording time
    for _ in range(num_segments):
        # Calculate the segment start index in data points (samples)
        segment_start_index = int(current_time * sampling_rate)
        segment_start_indices.append(segment_start_index)
        
        # Define the end of the current 5-minute segment in local recording time
        segment_end = current_time + 300  # 5 minutes in seconds

        # Select R-wave times within the current 5-minute segment
        segment_indices = (r_wave_times >= current_time) & (r_wave_times < segment_end)
        selected_r_wave_times = r_wave_times[segment_indices]
        segment_rr_intervals = np.diff(selected_r_wave_times) * 1000  # Convert RR intervals to milliseconds

        # Calculate HR and HRV if RR intervals are available
        if len(segment_rr_intervals) > 0:
            mean_hr = int(round(processing.calc_mean_hr(segment_rr_intervals / 1000, rr_units="seconds")))
            hrv = int(round(np.std(segment_rr_intervals)))
        else:
            mean_hr = np.nan
            hrv = np.nan

        # Append the HR and HRV results for each segment
        hr_values.append(mean_hr)
        hrv_values.append(hrv)

        # Move to the next 5-minute segment
        current_time = segment_end

    # Return HR, HRV values, QRS indices, and segment start points in sample indices
    return hr_values, hrv_values, qrs_indices, segment_start_indices


In [9]:
def transfer_to_final_structure(patient_id, hr_hrv_df, final_df):
    # Create a new row for the patient
    row_data = {"Patient_ID": patient_id}
    
    # Populate the row with HR and HRV values for each segment
    for i in range(len(hr_hrv_df)):
        hr_column = f"Segment_{i+1}_HR"
        hrv_column = f"Segment_{i+1}_HRV"
        
        row_data[hr_column] = hr_hrv_df.at[i, "Mean HR (bpm)"]
        row_data[hrv_column] = hr_hrv_df.at[i, "HRV (SDNN) (ms)"]
    
    # Convert row_data dictionary to a DataFrame and concatenate with final_df
    new_row_df = pd.DataFrame([row_data])
    final_df = pd.concat([final_df, new_row_df], ignore_index=True)
    return final_df

In [10]:
def get_patient_records(patient_number):
    # Construct the full URL to the RECORDS file
    file_url = f"{base_url}/{patient_number}/RECORDS"

    # Fetch the URL content
    response = requests.get(file_url)
    if response.status_code == 200:
        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the text content of the RECORDS file
        patient_records = soup.text  # Plain text response for RECORDS

        # Split the content into lines and filter for `_ECG` files with last 3 digits <= 023
        ecg_files = [
            line for line in patient_records.splitlines()
            if line.endswith("_ECG") and int(line.split("_")[-2]) <= 23
        ]

        return ecg_files
    else:
        print(f"Failed to retrieve the file. Status code: {response.status_code}")

In [ ]:
# List of ECG files for this patient
ecg_files = ["0284_001_004_ECG", "0284_002_005_ECG"]  # Adjust as needed

# Base directory path for the PhysioNet database
physionet_db_path = "i-care/2.1/training"

# Example patient number
patient_number = '0284'

# Iterate over each ECG file
for ecg_file in ecg_files:
    # Construct the file's record name (without extension)
    record_name = ecg_file
    
    try:
        # Load the record using PhysioNet-specific directory
        record = wfdb.rdrecord(record_name, pn_dir=f"{physionet_db_path}/{patient_number}")
        print(f"Successfully loaded: {ecg_file}")
    except Exception as e:
        print(f"Failed to load {ecg_file}: {e}")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): physionet.org:443
DEBUG:urllib3.connectionpool:https://physionet.org:443 "GET /files/i-care/2.1/training/0284/0284_001_004_ECG.hea HTTP/11" 200 174
INFO:wfdb.io._url:GET https://physionet.org/files/i-care/2.1/training/0284/0284_001_004_ECG.hea: 200 0-174/174
DEBUG:urllib3.connectionpool:https://physionet.org:443 "GET /files/i-care/2.1/training/0284/0284_001_004_ECG.mat HTTP/11" 206 3157000
INFO:wfdb.io._url:GET https://physionet.org/files/i-care/2.1/training/0284/0284_001_004_ECG.mat: 206 24-3157024/3157024
DEBUG:urllib3.connectionpool:https://physionet.org:443 "GET /files/i-care/2.1/training/0284/0284_002_005_ECG.hea HTTP/11" 200 174
INFO:wfdb.io._url:GET https://physionet.org/files/i-care/2.1/training/0284/0284_002_005_ECG.hea: 200 0-174/174


Successfully loaded: 0284_001_004_ECG
500


DEBUG:urllib3.connectionpool:https://physionet.org:443 "GET /files/i-care/2.1/training/0284/0284_002_005_ECG.mat HTTP/11" 206 3600000
INFO:wfdb.io._url:GET https://physionet.org/files/i-care/2.1/training/0284/0284_002_005_ECG.mat: 206 24-3600024/3600024


Successfully loaded: 0284_002_005_ECG
500


In [12]:
patient_numbers = ['0284']

# Download each patient's .txt file
for patient_number in patient_numbers:
    ecg_files = get_patient_records(patient_number)

    # Initialize a DataFrame for a full 24-hour period, with 5-minute segments
    total_duration_hours = 24
    segment_duration = timedelta(minutes=5)
    total_segments = (total_duration_hours * 60) // 5

    # Create the full DataFrame with NaN values initially
    hr_hrv_df = pd.DataFrame({
        "Start Time": [timedelta(minutes=5 * i) for i in range(total_segments)],
        "Mean HR (bpm)": [np.nan] * total_segments,
        "HRV (SDNN) (ms)": [np.nan] * total_segments
    })

    for ecg_file in ecg_files:

        # Construct the file's record name (without extension)
        record_name = ecg_file
        
        try:
            # Load the record using PhysioNet-specific directory
            record = wfdb.rdrecord(record_name, pn_dir=f"{physionet_db_path}/{patient_number}")
            print(f"Successfully loaded: {ecg_file}")
        except Exception as e:
            print(f"Failed to load {ecg_file}: {e}")

        # Get metadata of the signal
        patient_number = record.record_name.split("_")[0]
        sampling_rate = record.fs
        signal_len = record.sig_len
        utility_freq = int(record.comments[0].split(": ")[1])
        start_time = pd.to_timedelta(record.comments[1].split(": ")[1])
        end_time = pd.to_timedelta(record.comments[2].split(": ")[1])

        # Access the signal data (numpy array)
        ecg_signal_data = record.p_signal.flatten()

        filtered_signal = apply_high_low_filter(ecg_signal_data, sampling_rate, utility_freq)
        hr_values, hrv_values, qrs_indices, segment_start_points = calculate_hr_hrv(filtered_signal, sampling_rate, start_time, signal_len)

        # Find the nearest 5-minute interval for `start_time`
        start_index = hr_hrv_df[hr_hrv_df["Start Time"] <= start_time].last_valid_index()

        # Populate `hr_hrv_df` starting from `start_index`
        for i, (hr, hrv) in enumerate(zip(hr_values, hrv_values)):
            if start_index + i < len(hr_hrv_df):  # Ensure we don't go out of bounds
                hr_hrv_df.at[start_index + i, "Mean HR (bpm)"] = hr
                hr_hrv_df.at[start_index + i, "HRV (SDNN) (ms)"] = hrv


    # After processing all files for a patient, call this function
    final_df = transfer_to_final_structure(patient_number, hr_hrv_df, final_df)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): physionet.org:443
DEBUG:urllib3.connectionpool:https://physionet.org:443 "GET /content/i-care/2.1/training//0284/RECORDS HTTP/11" 200 None
DEBUG:urllib3.connectionpool:https://physionet.org:443 "GET /files/i-care/2.1/training/0284/0284_001_004_ECG.hea HTTP/11" 200 174
INFO:wfdb.io._url:GET https://physionet.org/files/i-care/2.1/training/0284/0284_001_004_ECG.hea: 200 0-174/174
DEBUG:urllib3.connectionpool:https://physionet.org:443 "GET /files/i-care/2.1/training/0284/0284_001_004_ECG.mat HTTP/11" 206 3157000
INFO:wfdb.io._url:GET https://physionet.org/files/i-care/2.1/training/0284/0284_001_004_ECG.mat: 206 24-3157024/3157024


Successfully loaded: 0284_001_004_ECG
500


DEBUG:urllib3.connectionpool:https://physionet.org:443 "GET /files/i-care/2.1/training/0284/0284_002_005_ECG.hea HTTP/11" 200 174
INFO:wfdb.io._url:GET https://physionet.org/files/i-care/2.1/training/0284/0284_002_005_ECG.hea: 200 0-174/174


First Segment Duration (s): 157.0
Remaining Time (s): 3000.0, Full 5-Minute Segments: 10
[100, 103, 104, 96, 96, 100, 101, 91, 92, 94, 102] [210, 36, 1, 98, 147, 72, 76, 110, 106, 98, 32]


DEBUG:urllib3.connectionpool:https://physionet.org:443 "GET /files/i-care/2.1/training/0284/0284_002_005_ECG.mat HTTP/11" 206 3600000
INFO:wfdb.io._url:GET https://physionet.org/files/i-care/2.1/training/0284/0284_002_005_ECG.mat: 206 24-3600024/3600024


...

Successfully loaded: 0284_023_022_ECG
500


DEBUG:urllib3.connectionpool:https://physionet.org:443 "GET /files/i-care/2.1/training/0284/0284_024_023_ECG.hea HTTP/11" 200 172
INFO:wfdb.io._url:GET https://physionet.org/files/i-care/2.1/training/0284/0284_024_023_ECG.hea: 200 0-172/172


First Segment Duration (s): 300.0
Remaining Time (s): 3300.0, Full 5-Minute Segments: 11
[98, 97, 97, 94, 95, 95, 93, 94, 90, 88, 89, 86] [144, 134, 141, 154, 153, 146, 151, 150, 164, 169, 177, 169]


DEBUG:urllib3.connectionpool:https://physionet.org:443 "GET /files/i-care/2.1/training/0284/0284_024_023_ECG.mat HTTP/11" 206 3600000
INFO:wfdb.io._url:GET https://physionet.org/files/i-care/2.1/training/0284/0284_024_023_ECG.mat: 206 24-3600024/3600024


Successfully loaded: 0284_024_023_ECG
500
First Segment Duration (s): 300.0
Remaining Time (s): 3300.0, Full 5-Minute Segments: 11
[86, 83, 82, 82, 83, 83, 83, 82, 81, 84, 88, 89] [169, 171, 182, 182, 182, 176, 176, 188, 181, 174, 157, 168]


In [13]:
import plotly.graph_objs as go
import plotly.offline as pyo

# Create the Plotly plot for the ECG Signal
trace_signal = go.Scatter(
    y=filtered_signal,
    mode='lines',
    name='ECG Signal'
)

# Create the Plotly plot for the R-Peaks
r_peaks_y = [filtered_signal[i] for i in qrs_indices]  # Get the amplitudes of the R-peaks
trace_r_peaks = go.Scatter(
    x=qrs_indices,
    y=r_peaks_y,
    mode='markers',
    name='R-Peaks',
    marker=dict(color='red', size=8, symbol='x')  # Customize the appearance
)

# Create vertical line markers at the start of each 5-minute segment
segment_lines = [
    go.layout.Shape(
        type="line",
        x0=boundary,
        y0=min(filtered_signal),  # Start from minimum signal amplitude
        x1=boundary,
        y1=max(filtered_signal),  # End at maximum signal amplitude
        line=dict(color="blue", width=1, dash="dash"),
    ) for boundary in segment_start_points
]

# Define the layout with added shapes
layout = go.Layout(
    title='Scrollable ECG Signal with R-Peaks and 5-Minute Segment Lines',
    xaxis=dict(
        title='Time (samples)',
        rangeslider=dict(visible=True),  # Add a range slider to make it scrollable
    ),
    yaxis=dict(
        title='Amplitude',
    ),
    shapes=segment_lines,  # Use segment_lines instead of shapes
    width=1200,  # Set a wider plot
    height=500
)

# Create the figure and add both traces
fig = go.Figure(data=[trace_signal, trace_r_peaks], layout=layout)

# Show the plot in an interactive window
pyo.plot(fig, filename='scrollable_ecg_with_r_peaks_segments.html')


'scrollable_ecg_with_r_peaks_segments.html'

In [14]:
final_df.to_csv('data/hr_hrv_df.csv', index=False)

In [15]:
print("Segment boundaries (in samples):", segment_start_points)

Segment boundaries (in samples): [0, 150000, 300000, 450000, 600000, 750000, 900000, 1050000, 1200000, 1350000, 1500000, 1650000]
